<a href="https://colab.research.google.com/github/chinhegde/Schedulize-Email-to-Calendar-Converter/blob/main/email_to_agenda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install simplegmail
! pip install icalendar
! pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 4.2 MB/s eta 0:00:00


In [ ]:
from simplegmail import Gmail

from icalendar import Calendar, Event
from datetime import datetime
import time

import json
from google.colab import userdata
import google.generativeai as genai

import logging

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s',
                    filename='app.log', filemode='a')

In [ ]:
try:
  gmail = Gmail()
  logging.info("Connected to GMail through simplegmail.")

except Exception as e:
  logging.error("Failed to initialize Gmail: %s", e)

In [ ]:
try:

  GEMINI_API_KEY=userdata.get('GOOGLE_API_KEY')
  genai.configure(api_key=GEMINI_API_KEY)

  logging.info("Configured genai with GEMINI_API_KEY successfully.")

except Exception as e:
  logging.error("Failed to configure genai: %s", e)

In [ ]:
def read_emails():

  try:
    messages = gmail.get_unread_inbox()
    logging.info("%d unread emails found.", len(messages))

    return messages

  except Exception as e:
    logging.error("Failed to initialize Gmail: %s", e)


In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
def extract_event_details(message_body):

  today = datetime.today().strftime("%m/%d/%Y")

  context = """Given the body an email - If the email mentions an event or provides an agenda, return a json format output - with event title, description, start and end dates (MM/DD/YYYY), start_time (HH:MM:SS), end_time (HH:MM:SS).
  Make sure that all the events mentioned in the body of the email are there in the JSON file. Alternatively, if there are no events or meetings mentioned in the email, return the string (not JSON) "NO_EVENTS_FOUND".
  - If time is mentioned relative to current day (example: today, tomorrow, etc.), use """ + today + """as today's date (MM/DD/YYYY).
  - If the end time is not provided, assume that the event is 30 minutes long only.
  - If no start or end times are mentioned (Example: Can we meet tomorrow? OR Event is day after tomorrow), set the time as 10 AM on the provided date.
  """

  user_input = message_body

  response = model.generate_content("System: " + context + "\nUser: " + user_input)

  # print(response.text)

  if response.text.strip() == '"NO_EVENTS_FOUND"':
    return

  events_data = json.loads(response.text[8:-4])

  return events_data


In [ ]:
unread_emails = read_emails()

4

In [ ]:
events_out = list()

In [ ]:
for email in unread_emails:

  time.sleep(5)

  print("Subject: " + email.subject)
  event = extract_event_details(email.plain)
  if event:
    events_out.append([email, event])
  else:
    continue

Subject: Annual Day Function Invitation - June 7th
** DEBUG 1 **
Subject: MIT Class of 2014 Reunion Events
** DEBUG 1 **
Subject: Invitation to High School Reunion on June 7th
** DEBUG 1 **
Subject: Coffee Chat on June 4th at 10:30 AM?
** DEBUG 1 **


In [ ]:
def create_calendar_event(events_list):

  cal = Calendar()
  event = Event()

  for e in events_list:

    try:
      e['event_title']
    except:
      return

    start_datetime_str = f"{e['start_date']} {e['start_time']}"
    start_datetime = datetime.strptime(start_datetime_str, "%m/%d/%Y %H:%M:%S")

    end_datetime_str = f"{e['end_date']} {e['end_time']}"
    end_datetime = datetime.strptime(end_datetime_str, "%m/%d/%Y %H:%M:%S")

    event.add('summary', e['event_title'])
    event.add('description', e['description'])
    event.add('dtstart', start_datetime)
    event.add('dtend', end_datetime)
    cal.add_component(event)

    return cal.to_ical()

In [ ]:
for info , events_list in events[:1]:

  # print(">>>>>", info.subject)

  cal_event = create_calendar_event(events_list)

  if cal_event:
    # print("Event exists.")

    with open('event.ics', 'wb') as f:
      f.write(cal_event)

    params = {
    "to": info.recipient,
    "sender": "chinmayi13hegde@gmail.com",
    "cc": [info.sender],
    "subject": "Calendar Invite: " + info.subject,
    "msg_html": "Calendar Invite",
    "msg_plain": "Hello,\nThis is an automated message to send a calendar invitation for the agenda specified in your email titled: " + info.subject,
    "attachments": ["/content/event.ics"],
    "signature": False  # use my account signature
    }

    message = gmail.send_message(**params)

>>>>> Annual Day Function Invitation - June 7th
Event exists.
